In [14]:
## 몇몇 명령어는 터미널에서만 수행해야하므로,
## 편의를 위해 모든 명령어는 터미널에서 수행합니다.
# 주석 노트
# - 코드가 길어지더라도 깨달은 것이나, 이전 버전 등은 주석 등으로 남기고 진행합니다.
#   따로 완벽하게 정리하려고 하면 진행이 너무 힘듭니다.
#   또한 주피터 자체가 하나의 노트입니다.

# conda를 업데이트 합니다. (선택사항)
conda update -y -n base -c defaults conda

# 이름이 <pororo>인 가상환경 제거 (재설치 하는 경우)
# - python 버전이 다른 경우 등의 상황에서 충돌이 발생하며, 재설치가 쉽고 빠릅니다.
conda deactivate                # 해당 환경이 활성화된 경우, 우선 비활성화합니다.
conda remove -y -n pororo --all # 설치된 패키지까지 모두 제거합니다.

conda env list # 가상환경 리스트 확인

# 이름이 <pororo>인 가상환경 생성
# - pororo 홈페이지에서는 "pororo only supports python>=3.6"라고 하지만,
#   정확히 python=3.6 버전이어야 합니다. (그냥 python으로 설치하면 충돌이 발생합니다.) 
# - torch 설치 부분에서 계속 멈췄는데, VSCode를 껐다 키는 것으로 해결하였습니다.
conda create -y -n pororo python=3.6 # python3.6 버전을 설치하면서 생성합니다.
conda activate pororo                # 생성된 환경을 활성화합니다.

# pororo 라이브러리 설치
# - marisa-trie 라이브러리를 찾을 수 없다는 에러가 발생하여, 따로 설치해주었습니다.
conda install -c conda-forge python-marisa-trie # marisa-trie 라이브러리 설치
pip install pororo # pororo 라이브러리 설치

# 가상환경을 커널로 설치
# - 터미널에서는 conda activate가 잘 작동하지만, 주피터에서는 작동하지 않습니다.
# - 주피터에서는 커널을 선택하며, 해당 커널은 설치된 가상환경만 사용합니다. (?)
# - 현재 활성화된 가상환경에서 커널 설치를 수행하는 방식으로 커널 생성을 수행합니다.
conda install ipykernel # 우선 해당 환경에 ipykernel 라이브러리를 설치합니다.
ipython kernel install --user --name=pororo # <pororo>라는 이름으로 커널을 생성합니다.

# 커널 리스트 확인: jupyter kernelspec list
# 이름이 <pororo>인 커널 삭제: jupyter kernelspec uninstall pororo
## 커널 생성 후, VSCode 자체를 재실행해야 적용됩니다 

/bin/bash: /opt/ml/anaconda3/bin/activate: No such file or directory


In [ ]:
## 수행 속도가 지나치게 느렸습니다. 이런 경우, GPU를 사용하고 있지 않을 확률이 큽니다.
#  - nvidia-smi 커맨드를 이용하여 cuda 버전 등을 확인할 수 있습니다.
#    cuda가 없으면 cuda 드라이버를 설치합니다.
#  - torch.cuda.is_available()로 cuda가 사용가능한지 확인합니다.

# 이번 경우에는 cuda 드라이버는 설치되어 있었지만, 사용은 불가능했습니다.
# - torchvision 및 cudatoolkit을 재설치하여 해결하였습니다.
#   conda uninstall -y torchvision
#   conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
import torch

!nvidia-smi
torch.cuda.is_available()

# 이후 GPU를 사용하고 있는지 터미널에서 확인합니다.
# - watch -n 1 nvidia-smi (터미널에 입력하며, 1초마다 확인합니다.)
#   제대로 작동함을 확인하였으며, 계산 속도가 개선되었습니다.
# - 종료하려면 ctrl+z 혹은 crtl+c를 누릅니다.
#   ctrl+z: stop(suspend)
#   ctrl+c: exit(kill)

In [3]:
from pororo import Pororo

# machine translation model
mt = Pororo(task="translation", lang="multi")

# example
org = "케빈은 아직도 일을 하고 있다."
trn = mt(org, src="ko", tgt="en")
mt(trn, src="en", tgt="ko")

'케빈은 아직 일하고 있다'

In [4]:
# translation-retranslation function
def get_retranslated(org: str) -> str:
    return mt(mt(org, src="ko", tgt="en"), src="en", tgt="ko")

In [20]:
#!conda install -y pandas

In [5]:
import pandas as pd
pd.options.display.max_colwidth = 68

# data load
data_path = "/opt/ml/input/data/train/train.tsv"
org = pd.read_csv(data_path, delimiter="\t", header=None)
org[1].head(3)
#org.loc[:, 1:]

0    영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)와 지프(Jeep)가 있으며, 이...
1    선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석, 비례대표 30석)을 획득하는 데...
2    유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 2017년 대회부터 UEFA U-...
Name: 1, dtype: object

In [59]:
#!conda install -y tqdm

In [29]:
from tqdm import tqdm

# augmentation
aug = org.copy(deep=True)
for i, s in enumerate(tqdm(aug[1])):
    aug[1][i] = get_retranslated(s)
aug[1].head(3)


In [23]:
# original DF에 augmentation DF를 append합니다.
# - ignore_index하여 augmentation의 index가 original 뒤에 이어지도록 합니다.
tot = org.append(aug, ignore_index=True)[:12000]
tot[1][9000:9003]

9000    영국에서 사용되는 스포츠카 브랜드에는 랜드로버와 지프가 포함되어 있으며, 이들 브랜딩도 자동차 유형을 언급하는 단어...
9001                선거에서는 민주당이 해산 전 230석을 훨씬 밑도는 57석(구 27석, 비례대표 30석)에 그쳤다.
9002    유럽축구연맹(UEFA) 집행위원회는 2017년 1월 24일 열린 회의를 통해 UEFFA U-21 축구대회 참가국을 ...
Name: 1, dtype: object

In [28]:
# 저장
# 대회에서 주어진 원본 데이터 형식에 따릅니다.
outdata_path = "/opt/ml/input/data/train/train_aug.tsv"
tot.to_csv(outdata_path, index=False, header=False, sep="	")